In [1]:
from torch.distributions import Categorical
from torch import nn
import torch
import gym
from collections import deque
import itertools
import numpy as np
import random
import torch.nn.functional as F
from datetime import datetime
import matplotlib.pyplot as plt
import cv2


In [2]:
MAX_REWARD_REACH = -100
BATCH_SIZE=256
GAMMA = 0.99 #discount factor for future utilities
# env = gym.make('CartPole-v1') #Make environment
NUM_EPISODES = 3000 #number of episodes to run
MAX_STEPS = 10000 #max steps per episode
SOLVED_SCORE = 195 #score agent needs for environment to be solved
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" #device to run model on 
Model_Name = "Reinforce_Base_line"
LR = 1e-3

ENV_Name='Acrobot-v1'

In [3]:
torch.manual_seed(2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
env = gym.make(ENV_Name)
env.seed(0)
env.env.book_or_nips = 'nips'

/home/ragu_robo/.local/lib/python3.8/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/ragu_robo/.local/lib/python3.8/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/ragu_robo/.local/lib/python3.8/site-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


In [4]:
def select_action(network, state):
    ''' Selects an action given current state
    Args:
    - network (Torch NN): network to process state
    - state (Array): Array of action space in an environment
    
    Return:
    - (int): action that is selected
    - (float): log probability of selecting that action given state and network
    '''
    
    #convert state to float tensor, add 1 dimension, allocate tensor on device
    state = torch.from_numpy(state).float().unsqueeze(0).to(DEVICE)
    
    #use network to predict action probabilities
    action_probs = network(state)
    state = state.detach()
    
    #sample an action using the probability distribution
    m = Categorical(action_probs)
    action = m.sample()
    
    #return action
    return action.item(), m.log_prob(action)

In [5]:
#Using a neural network to learn our policy parameters
class PolicyNetwork(nn.Module):
    
    #Takes in observations and outputs actions
    def __init__(self, observation_space, action_space):
        super(PolicyNetwork, self).__init__()
        self.input_layer = nn.Linear(observation_space, 128)
        self.output_layer = nn.Linear(128, action_space)
    
    #forward pass
    def forward(self, x):
        #input states
        x = self.input_layer(x)
        
        #relu activation
        x = F.relu(x)
        
        #actions
        actions = self.output_layer(x)
        
        #get softmax for a probability distribution
        action_probs = F.softmax(actions, dim=1)
        
        return action_probs

In [6]:
# import gym
# import cv2

# # Define your environment
# env_name = 'CartPole-v1'
# env = gym.make(env_name)

# # Reset the environment
# observation = env.reset()

# # Define video writer
# video_path = 'output.avi'
# fps = 30.0
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(video_path, fourcc, fps, (600, 400))  # Change resolution as needed

# done = False
# for c in range(1000):
#     # Render the environment
#     frame = env.render(mode='rgb_array')

#     # Convert the frame to BGR for OpenCV
#     frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

#     # Write frame to the video
#     out.write(frame_bgr)

#     # Take an action
#     action = env.action_space.sample()  # Random action for demonstration
#     observation, reward, done, info = env.step(action)

# # Release video writer and close environment
# out.release()
# env.close()

In [7]:
path = f'./All_ckpt/iteration_4_{ENV_Name}_{Model_Name}_best_checkpoint__.pth'

online_net = PolicyNetwork(env.observation_space.shape[0], env.action_space.n).to(DEVICE)

pretrained_statedict = torch.load(path)
online_net.load_state_dict(pretrained_statedict["state_dict"])



<All keys matched successfully>

In [8]:
# Define video writer
video_path = f'Q1Acrobot{Model_Name}.mp4'
fps = 30.0
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use mp4v codec for mp4 files
out = cv2.VideoWriter(video_path, fourcc, fps, (500, 500))  # Change resolution as needed

state = env.reset() 
for c in range(300):
    frame = env.render(mode='rgb_array')
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    print(frame_bgr.shape)
    out.write(frame_bgr)

#     # Take an action
#     env.render()
    # Random action for demonstration
#     state = torch.as_tensor(state, dtype=torch.float32).to(device)
    action, lp = select_action(online_net, state)

#     action = online_net.act(state)
    state, reward, done, info = env.step(action)
    
    if done:
        state = env.reset() 

        
    

#     env.render()

out.release()
env.close()

/home/ragu_robo/.local/lib/python3.8/site-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 500, 3)
(500, 